
<a href="https://colab.research.google.com/github/google/seqio/blob/main/seqio/notebooks/Basics_Task_and_Mixtures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Installing dependencies...")
!pip install seqio-nightly

import seqio
import tensorflow as tf
import functools

Installing dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.1/352.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 545.7/545.7 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 79.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 37.5 MB/s eta 0:00:00
  Created wheel for ml-collections: filename=ml_colle

# s1. define simplest seqio.Task

Let’s define the simplest SeqIO Task that just reads data from a TFDS dataset, no preprocessing.

In [2]:
seqio.TaskRegistry.add(
   'my_simple_task',
   source=seqio.TfdsDataSource('natural_questions_open:1.0.0'),
   output_features={}
)


then we get the task from the registry, get the dataset from the task, and see a batch of data.

In [3]:
task = seqio.TaskRegistry.get('my_simple_task')
ds = task.get_dataset(sequence_length=None, split="train", shuffle=False)
list(ds.take(1).as_numpy_iterator())

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/87925 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/natural_questions_open/1.0.0.incomplete2I6E1A/natural_questions_open-train…

Generating validation examples...:   0%|          | 0/3610 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/natural_questions_open/1.0.0.incomplete2I6E1A/natural_questions_open-valid…

Dataset natural_questions_open downloaded and prepared to /root/tensorflow_datasets/natural_questions_open/1.0.0. Subsequent calls will reuse this data.


[{'answer': array([b'Romi Van Renterghem.'], dtype=object),
  'question': b'who is the girl in more than you know'}]

#s2. add preprocessors

To make it more ready for sequence modeling, we need change a few things.

- We'd like data batch to have keys `inputs` and `targets` instead of `question` and `answer` to make the naming less task-dependent. This can be done by `seqio.preprocessors.rekey()`.

- The `answer` field is currently a list of sequences (texts), whereas modeling often assumes there's only one output sequence. We need to sample a single sequence from the list.

- We need to tokenize both `inputs` and `targets`, for which we can use `seqio.preprocessors.tokenize` and supply a seqio vocabulary.

## s2.1 add rekey

In [4]:
seqio.TaskRegistry.remove('my_simple_task')
seqio.TaskRegistry.add(
    'my_simple_task',
    source=seqio.TfdsDataSource('natural_questions_open:1.0.0'),
    preprocessors=[
       functools.partial(
           seqio.preprocessors.rekey,
           key_map={
               'inputs': 'question',
               'targets': 'answer',
           }),
   ],
    output_features={}
)

we check the same batch of the data and will see the keys are changed to `inputs` and `targets`.

In [5]:
task = seqio.TaskRegistry.get('my_simple_task')
ds = task.get_dataset(sequence_length=None, split="train", shuffle=False)
list(ds.take(1).as_numpy_iterator())

[{'inputs': b'who is the girl in more than you know',
  'targets': array([b'Romi Van Renterghem.'], dtype=object)}]

## s2.2 sample one sequence from answers/targets

Here we need to define our own preprocessor function using `seqio.map_over_dataset`.

In [6]:
# seqio.map_over_dataset is decorator to map decorated function
# (e.g., sample_from_answers below) over all examples in a dataset.
# for details, please refer to seqio.map_over_dataset() documentation.
@seqio.map_over_dataset(num_seeds=1)
def sample_from_answers(x, seed):
 answers = x['targets']
 sample_id = tf.random.stateless_uniform([],
                                         seed=seed,
                                         minval=0,
                                         maxval=len(answers),
                                         dtype=tf.int32)
 x['targets'] = answers[sample_id]
 return x


the preprocessor `sample_from_answers` can be added after `seqio.preprocessors.rekey` so that seqio will execute them in the order listed.

In [7]:
seqio.TaskRegistry.remove('my_simple_task')
seqio.TaskRegistry.add(
    'my_simple_task',
    source=seqio.TfdsDataSource('natural_questions_open:1.0.0'),
    preprocessors=[
       functools.partial(
           seqio.preprocessors.rekey,
           key_map={
               'inputs': 'question',
               'targets': 'answer',
           }),
       sample_from_answers,
   ],
    output_features={}
)

we check the same batch of the data and should see there's only one string in the `targets` field.

In [8]:
task = seqio.TaskRegistry.get('my_simple_task')
ds = task.get_dataset(sequence_length=None, split="train", shuffle=False)
list(ds.take(1).as_numpy_iterator())

Instructions for updating:
Use `tf.data.Dataset.random(...)`.


[{'inputs': b'who is the girl in more than you know',
  'targets': b'Romi Van Renterghem.'}]

## s2.3 tokenize `inputs` and `targets`

We will use a common vocabulary `SentencePieceVocabulary` here to tokenize the sequences.

In [9]:
sentencepiece_model_file = "gs://t5-data/vocabs/cc_all.32000.100extra/sentencepiece.model"
vocab = seqio.SentencePieceVocabulary(sentencepiece_model_file)

In [10]:
seqio.TaskRegistry.remove('my_simple_task')
seqio.TaskRegistry.add(
    'my_simple_task',
    source=seqio.TfdsDataSource('natural_questions_open:1.0.0'),
    preprocessors=[
       functools.partial(
           seqio.preprocessors.rekey,
           key_map={
               'inputs': 'question',
               'targets': 'answer',
           }),
       sample_from_answers,
       seqio.preprocessors.tokenize,
   ],
    output_features={
        'inputs': seqio.Feature(vocabulary=vocab),
        'targets': seqio.Feature(vocabulary=vocab),
    }
)

we check the same batch of the data and should see the `inputs` and `targets` become tokenized sequences (arrays of integers). We also keep the original `inputs` and `targets` in `inputs_pretokenized` and `targets_pretokenized` fields.

In [11]:
task = seqio.TaskRegistry.get('my_simple_task')
ds = task.get_dataset(sequence_length=None, split="train", shuffle=False)
list(ds.take(1).as_numpy_iterator())

[{'inputs_pretokenized': b'who is the girl in more than you know',
  'inputs': array([ 113,   19,    8, 3202,   16,   72,  145,   25,  214], dtype=int32),
  'targets_pretokenized': b'Romi Van Renterghem.',
  'targets': array([12583,    23,  4480,  9405,    49,   122,  6015,     5],
        dtype=int32)}]

# s3. define a Mixture

To make a seqio Mixture, we need at least two tasks. So let's define another one same as `my_simple_task` but name it as `my_simple_task2`.

In [12]:
seqio.TaskRegistry.remove('my_simple_task2')
seqio.TaskRegistry.add(
    'my_simple_task2',
    source=seqio.TfdsDataSource('natural_questions_open:1.0.0'),
    preprocessors=[
       functools.partial(
           seqio.preprocessors.rekey,
           key_map={
               'inputs': 'question',
               'targets': 'answer',
           }),
       sample_from_answers,
       seqio.preprocessors.tokenize,
   ],
    output_features={
        'inputs': seqio.Feature(vocabulary=vocab),
        'targets': seqio.Feature(vocabulary=vocab),
    }
)

In [13]:
seqio.MixtureRegistry.add(
    'my_simple_mixture',
    [('my_simple_task', 0.5), 'my_simple_task2'],
    default_rate=1.0
)

In [14]:
mixture = seqio.MixtureRegistry.get('my_simple_mixture')
ds = mixture.get_dataset(sequence_length=None, split="train", shuffle=False)
list(ds.take(6).as_numpy_iterator())

[{'inputs': array([ 113,   19,    8, 3202,   16,   72,  145,   25,  214], dtype=int32),
  'targets': array([12583,    23,  4480,  9405,    49,   122,  6015,     5],
        dtype=int32)},
 {'inputs': array([  116,   405,   467,   147,   388,   369,    91,    30,  3134,
            89, 17591], dtype=int32),
  'targets': array([ 1332, 12992,   846], dtype=int32)},
 {'inputs': array([ 116,   47,    8,    3, 5397,   15,   13,    8, 4913, 8769,   52,
         1545], dtype=int32),
  'targets': array([1003, 4327,  104, 3916], dtype=int32)},
 {'inputs': array([11003,    13,    16,  8603,  2717,    16,  1353,    13, 19680,
             3,   122,    26,   102], dtype=int32),
  'targets': array([17353,    18, 15599,     7,    17], dtype=int32)},
 {'inputs': array([  113,    47,     8,   163,   178,  2753,   113,     7, 18573,
            47,    92,  2753], dtype=int32),
  'targets': array([15717, 20429], dtype=int32)},
 {'inputs': array([ 113, 2832,    3,   23,  317,    3,   23,   31,   51,  352,

# s4. add feature converters

Up to this point, both `Task` and `Mixture` are model-agnostic data processing. Feature converter provides a way to introduce model-specific data processing. The example feature converter below is used for encoder-decoder modeling, called `seqio.EncDecFeatureConverter`.

In [15]:
fc = seqio.EncDecFeatureConverter(pack=True)
feature_lengths = {"inputs": 5, "targets": 5}
# Truncate inputs or targets to the length specified in feature_lengths.
# And the truncation happens after tokenization.
task = seqio.TaskRegistry.get('my_simple_task')
ds = task.get_dataset(sequence_length=feature_lengths, split="train", shuffle=False)
list(ds.take(1).as_numpy_iterator())

[{'inputs_pretokenized': b'who is the girl in more than you know',
  'inputs': array([ 113,   19,    8, 3202,   16], dtype=int32),
  'targets_pretokenized': b'Romi Van Renterghem.',
  'targets': array([12583,    23,  4480,  9405,    49], dtype=int32)}]

we check the same batch and will see it has more fields that are needed for encoder-decoder modeling.

In [16]:
ds = fc(ds, feature_lengths)
list(ds.take(1).as_numpy_iterator())

[{'encoder_input_tokens': array([ 113,   19,    8, 3202,   16], dtype=int32),
  'decoder_target_tokens': array([12583,    23,  4480,  9405,    49], dtype=int32),
  'decoder_input_tokens': array([    0, 12583,    23,  4480,  9405], dtype=int32),
  'decoder_loss_weights': array([1, 1, 1, 1, 1], dtype=int32),
  'encoder_segment_ids': array([1, 1, 1, 1, 1], dtype=int32),
  'decoder_segment_ids': array([1, 1, 1, 1, 1], dtype=int32),
  'encoder_positions': array([0, 1, 2, 3, 4], dtype=int32),
  'decoder_positions': array([0, 1, 2, 3, 4], dtype=int32)}]